In [0]:
!unzip house-prices-advanced-regression-techniques.zip

Archive:  house-prices-advanced-regression-techniques.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler

In [0]:
import pandas as pd

'''
データのロード
'''
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [0]:
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [22]:
# trainとtestのラベルを追加
train['Type'] = 'train'
test['Type'] = 'test'
data = train.append(test)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [0]:
'''
データのクリーニング
'''

'\nデータのクリーニング\n'

In [0]:
def missing_table(df):      # データの欠損を計算
    null_val = df.isnull().sum()
    percent = 100 * df.isnull().sum()/len(df)
    missing_table = pd.concat([null_val, percent], axis=1)
    missing_table_ren_columns = missing_table.rename(
        columns = {0:"欠損数", 1:"%"}
    )
    return missing_table_ren_columns

In [0]:
def learge_missing(rate):
  index = []
  name = []
  null_val = data.isnull().sum()
  percent = 100 * data.isnull().sum()/len(data)
  for i in range(len(percent)):
    if percent[i] > rate:
      index.append(i)
  _ = null_val[index]
  name.append(_.index)
#   print(100 * data[list(np.array(name).flatten())].isnull().sum()/len(data[list(np.array(name).flatten())]))
  return list(np.array(name).flatten())

In [0]:
learge_missing(30)

['Alley', 'Fence', 'FireplaceQu', 'MiscFeature', 'PoolQC', 'SalePrice']

In [0]:
# Nanをラベルにしているコラムの調整
titles = learge_missing(30)[:-1]
for title in titles:
  data[title].fillna(0, inplace = True)

In [0]:
# 欠損値の補完
miss_title = [s for s in learge_missing(0) if 'SalePrice' not in s]
for title in miss_title:
  data[title].fillna(data[title].mode()[0], inplace = True)

In [0]:
objects = data.select_dtypes(include=object)
# objects.nunique()

In [0]:
# コラムのエンコード

target_col = ["SalePrice"]
id_dataset = ["Type"]
cat_cols   = data.nunique()[data.nunique() < 26].keys().tolist()   # unique数が26未満のラベルをcat_colsにまとめる
cat_cols   = [x for x in cat_cols ]
# numerical columns(数値)
num_cols   = [x for x in data.columns if x not in cat_cols + target_col + id_dataset]  # cat_cols, target_col, id_datasetのどれにも属さないラベルをまとめる　
# Binary columns with 2 values(バイナリ)
bin_cols   = data.nunique()[data.nunique() == 2].keys().tolist()
# Columns more than 2 values(3種以上のクラス)
multi_cols = [i for i in cat_cols if i not in bin_cols]

In [0]:
# bin_colsのエンコード
le = LabelEncoder()
for i in bin_cols :
    data[i] = le.fit_transform(data[i])

In [0]:
# multi_colsのダミーデータを生成
data = pd.get_dummies(data = data,columns = multi_cols )

In [0]:
# # num_colsを標準化
# sc = RobustScaler()
# scaled = sc.fit_transform(data[num_cols])
# scaled = pd.DataFrame(scaled,columns = num_cols)

In [0]:
# # 不要なラベルの削除とscaledのmerge
# df_data_og = data.copy()
# data = data.drop(columns = num_cols,axis = 1)
# data = data.merge(scaled,left_index = True,right_index = True,how = "left")

In [0]:
# dataの分離
train = data[data['Type'] == 1].drop(columns = ['Type'])
test = data[data['Type'] == 0].drop(columns = ['Type'])

In [30]:
# trainを入力と正解に分離しarrayに変換
X_train = train.iloc[:, 1:].as_matrix()
Y_train = train.iloc[:,0].as_matrix()
X_test = test.iloc[:, 1:].as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
'''
データの保存
'''
import pickle

pickle_file = "./HousePrice_data_ns.pickle"
with open(pickle_file, 'wb') as f:
    pickle.dump(X_train, f)
    pickle.dump(Y_train, f)
    pickle.dump(X_test, f)

In [0]:
with open(pickle_file, 'rb') as f:
    X_train = pickle.load(f)
    Y_train = pickle.load(f)
    X_test = pickle.load(f)